## Get the Data

Either use the provided .csv file or (optionally) get fresh (the freshest?) data from running an SQL query on StackExchange: 

Follow this link to run the query from [StackExchange](https://data.stackexchange.com/stackoverflow/query/675441/popular-programming-languages-per-over-time-eversql-com) to get your own .csv file

<code>
select dateadd(month, datediff(month, 0, q.CreationDate), 0) m, TagName, count(*)
from PostTags pt
join Posts q on q.Id=pt.PostId
join Tags t on t.Id=pt.TagId
where TagName in ('java','c','c++','python','c#','javascript','assembly','php','perl','ruby','visual basic','swift','r','object-c','scratch','go','swift','delphi')
and q.CreationDate < dateadd(month, datediff(month, 0, getdate()), 0)
group by dateadd(month, datediff(month, 0, q.CreationDate), 0), TagName
order by dateadd(month, datediff(month, 0, q.CreationDate), 0)
</code>

## Import Statements

In [1]:
import pandas as pd

## Data Exploration

**Challenge**: Read the .csv file and store it in a Pandas dataframe

In [9]:
dataframe = pd.read_csv('QueryResults_2024.csv', names=['DATE', 'TAG', 'POSTS'], header=0)

**Challenge**: Examine the first 5 rows and the last 5 rows of the of the dataframe

In [10]:
dataframe.head()

,DATE,TAG,POSTS
0,2008-07-01 00:00:00,c#,3
1,2008-08-01 00:00:00,ruby,70
2,2008-08-01 00:00:00,perl,28
3,2008-08-01 00:00:00,python,119
4,2008-08-01 00:00:00,delphi,13


In [11]:
dataframe.tail()

,DATE,TAG,POSTS
2616,2024-04-01 00:00:00,go,323
2617,2024-04-01 00:00:00,r,1566
2618,2024-04-01 00:00:00,delphi,109
2619,2024-04-01 00:00:00,php,1281
2620,2024-04-01 00:00:00,ruby,117


**Challenge:** Check how many rows and how many columns there are. 
What are the dimensions of the dataframe?

In [12]:
dataframe.shape

(2621, 3)

**Challenge**: Count the number of entries in each column of the dataframe

In [13]:
dataframe.count()

DATE     2621
TAG      2621
POSTS    2621
dtype: int64

**Challenge**: Calculate the total number of post per language.
Which Programming language has had the highest total number of posts of all time?

In [15]:
dataframe.groupby("TAG").sum()

,DATE,POSTS
TAG,,
assembly,2008-08-01 00:00:002008-09-01 00:00:002008-10-...,44328
c,2008-08-01 00:00:002008-09-01 00:00:002008-10-...,404167
c#,2008-07-01 00:00:002008-08-01 00:00:002008-09-...,1615549
c++,2008-08-01 00:00:002008-09-01 00:00:002008-10-...,806887
delphi,2008-08-01 00:00:002008-09-01 00:00:002008-10-...,51907
go,2009-11-01 00:00:002009-12-01 00:00:002010-01-...,72931
java,2008-08-01 00:00:002008-09-01 00:00:002008-10-...,1916608
javascript,2008-08-01 00:00:002008-09-01 00:00:002008-10-...,2527275
perl,2008-08-01 00:00:002008-09-01 00:00:002008-10-...,68100


Some languages are older (e.g., C) and other languages are newer (e.g., Swift). The dataset starts in September 2008.

**Challenge**: How many months of data exist per language? Which language had the fewest months with an entry? 


In [16]:
dataframe.groupby("TAG").count()

,DATE,POSTS
TAG,,
assembly,189,189
c,189,189
c#,190,190
c++,189,189
delphi,189,189
go,174,174
java,189,189
javascript,189,189
perl,189,189


## Data Cleaning

Let's fix the date format to make it more readable. We need to use Pandas to change format from a string of "2008-07-01 00:00:00" to a datetime object with the format of "2008-07-01"

In [19]:
print(dataframe["DATE"][1])
print(type(dataframe.DATE[1]))

2008-08-01 00:00:00
<class 'str'>


In [21]:
print(pd.to_datetime(dataframe['DATE'][1]))
print(type(pd.to_datetime(dataframe['DATE'][1])))

2008-08-01 00:00:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [23]:
dataframe.DATE = pd.to_datetime(dataframe.DATE)
dataframe.head()

,DATE,TAG,POSTS
0,2008-07-01,c#,3
1,2008-08-01,ruby,70
2,2008-08-01,perl,28
3,2008-08-01,python,119
4,2008-08-01,delphi,13


## Data Manipulation



Sometimes you want to convert your DataFrame so that each category has its own column.
The easiest way to accomplish this is by using the .pivot() method in Pandas.

In [25]:
pivoted_df = dataframe.pivot(index='DATE', columns='TAG', values='POSTS')

**Challenge**: What are the dimensions of our new dataframe? How many rows and columns does it have? Print out the column names and print out the first 5 rows of the dataframe.

In [26]:
pivoted_df.shape

(190, 14)

In [32]:
pivoted_df.columns

Index(['assembly', 'c', 'c#', 'c++', 'delphi', 'go', 'java', 'javascript',
       'perl', 'php', 'python', 'r', 'ruby', 'swift'],
      dtype='object', name='TAG')

In [29]:
pivoted_df.head()

TAG,assembly,c,c#,c++,delphi,go,java,javascript,perl,php,python,r,ruby,swift
DATE,,,,,,,,,,,,,,
2008-07-01,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-08-01,8.0,83.0,503.0,164.0,13.0,NaN,222.0,163.0,28.0,157.0,119.0,NaN,70.0,NaN
2008-09-01,28.0,320.0,1639.0,755.0,104.0,NaN,1129.0,634.0,130.0,474.0,536.0,6.0,286.0,NaN
2008-10-01,16.0,303.0,1987.0,807.0,112.0,NaN,1148.0,722.0,128.0,604.0,506.0,NaN,246.0,NaN
2008-11-01,16.0,258.0,1729.0,734.0,140.0,NaN,954.0,580.0,97.0,497.0,448.0,1.0,157.0,NaN


**Challenge**: Count the number of entries per programming language. Why might the number of entries be different? 

In [33]:
pivoted_df.count()

TAG
assembly      189
c             189
c#            190
c++           189
delphi        189
go            174
java          189
javascript    189
perl          189
php           189
python        189
r             187
ruby          189
swift         180
dtype: int64

### The .count() method excludes NaN values.

## Dealing with NaN Values

In this case, we don't want to drop the rows that have a NaN value. Instead, we want to substitute the number 0 for each NaN value in the DataFrame. We can do this with the .fillna() method. 

In [35]:
pivoted_df.fillna(0, inplace=True)
pivoted_df.head()

TAG,assembly,c,c#,c++,delphi,go,java,javascript,perl,php,python,r,ruby,swift
DATE,,,,,,,,,,,,,,
2008-07-01,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-08-01,8.0,83.0,503.0,164.0,13.0,0.0,222.0,163.0,28.0,157.0,119.0,0.0,70.0,0.0
2008-09-01,28.0,320.0,1639.0,755.0,104.0,0.0,1129.0,634.0,130.0,474.0,536.0,6.0,286.0,0.0
2008-10-01,16.0,303.0,1987.0,807.0,112.0,0.0,1148.0,722.0,128.0,604.0,506.0,0.0,246.0,0.0
2008-11-01,16.0,258.0,1729.0,734.0,140.0,0.0,954.0,580.0,97.0,497.0,448.0,1.0,157.0,0.0


**Checking if there is any more NaN value in the dataframe**

In [37]:
pivoted_df.isna().values.any()

False

## Data Visualisaton with with Matplotlib


**Challenge**: Use the [matplotlib documentation](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot) to plot a single programming language (e.g., java) on a chart.

In [38]:
import matplotlib.pyplot as plt

**Challenge**: Show two line (e.g. for Java and Python) on the same chart.

# Smoothing out Time Series Data

Time series data can be quite noisy, with a lot of up and down spikes. To better see a trend we can plot an average of, say 6 or 12 observations. This is called the rolling mean. We calculate the average in a window of time and move it forward by one overservation. Pandas has two handy methods already built in to work this out: [rolling()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) and [mean()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.window.rolling.Rolling.mean.html). 